# **Distribución % de créditos por segmento (trimestre)**

In [4]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "tulancingo" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_tulancingo',
 'df_financiamientos_2020_tulancingo',
 'df_financiamientos_2021_tulancingo',
 'df_financiamientos_2022_tulancingo',
 'df_financiamientos_2023_tulancingo']

In [5]:
df_financiamientos_2022_tulancingo = df_financiamientos_2022_tulancingo[['id','mes','monto']]
df_financiamientos_2023_tulancingo = df_financiamientos_2023_tulancingo[['id','mes','monto']]

## 2022

In [6]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
df_financiamientos_2022_tulancingo['trimestre'] = df_financiamientos_2022_tulancingo['mes'].apply(asignar_trimestre)
df_financiamientos_2022_tulancingo

,id,mes,monto,trimestre
0,1,1,0.00,1
1,2,1,0.00,1
2,3,1,14398.00,1
3,4,1,96283.00,1
4,5,1,159720.57,1
...,...,...,...,...
311,312,12,2279250.00,4
312,313,12,2285795.72,4
313,314,12,2661350.00,4
314,315,12,2946484.00,4


In [7]:
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1750000.00),
    'C': (1750001.00, 2500000.00),
    'B': (2500001.00, 3250000.00),
    'A': (3250001.00, 4000000.00),
    'S': (4000001.00, float('inf'))  # Segmento S con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2022_tulancingo['segmento'] = df_financiamientos_2022_tulancingo['monto'].apply(asignar_segmento)
df_financiamientos_2022_tulancingo.head()

,id,mes,monto,trimestre,segmento
0,1,1,0.00,1,E
1,2,1,0.00,1,E
2,3,1,14398.00,1,E
3,4,1,96283.00,1,E
4,5,1,159720.57,1,E


In [8]:
df_financiamientos_2022_tulancingo = df_financiamientos_2022_tulancingo[['trimestre','segmento']]
df_financiamientos_2022_tulancingo

,trimestre,segmento
0,1,E
1,1,E
2,1,E
3,1,E
4,1,E
...,...,...
311,4,C
312,4,C
313,4,B
314,4,B


In [12]:
# Calcular el total de registros
total_registros = len(df_financiamientos_2022_tulancingo)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(df_financiamientos_2022_tulancingo, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  316
trimestre          1          2          3          4
segmento                                             
A           0.316456   0.000000   0.316456   0.000000
B           0.632911   0.949367   0.316456   0.632911
C           1.265823   1.265823   0.949367   2.531646
D           4.430380   5.379747   7.278481   4.430380
E          18.037975  12.658228  19.936709  18.037975
S           0.000000   0.316456   0.000000   0.316456


In [14]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
#porcentajes = tabla_porcentajes[trimestre]
fig = go.Figure()
trimestres = tabla_porcentajes.columns
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=tabla_porcentajes[trimestre],  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por segmento (trimestre) 2022',
    yaxis=dict(
        #tickformat='2.2%',
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    
    legend_title="Trimestre",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_%_2022',  carpeta='assets/graficas')
fig.show()

## 2023

In [15]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
df_financiamientos_2023_tulancingo['trimestre'] = df_financiamientos_2023_tulancingo['mes'].apply(asignar_trimestre)
df_financiamientos_2023_tulancingo

,id,mes,monto,trimestre
0,1,1,1585485.28,1
1,2,1,608908.05,1
2,3,1,1918694.86,1
3,4,1,1241000.00,1
4,5,2,0.00,1
5,6,2,390087.63,1
6,7,2,604900.00,1
7,8,2,1429200.00,1
8,9,2,1900000.00,1
9,10,2,1100000.00,1


In [16]:
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1750000.00),
    'C': (1750001.00, 2500000.00),
    'B': (2500001.00, 3250000.00),
    'A': (3250001.00, 4000000.00),
    'S': (4000001.00, float('inf'))  # Segmento S con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2023_tulancingo['segmento'] = df_financiamientos_2023_tulancingo['monto'].apply(asignar_segmento)
df_financiamientos_2023_tulancingo.head()

,id,mes,monto,trimestre,segmento
0,1,1,1585485.28,1,D
1,2,1,608908.05,1,E
2,3,1,1918694.86,1,C
3,4,1,1241000.00,1,D
4,5,2,0.00,1,E


In [17]:
df_financiamientos_2023_tulancingo = df_financiamientos_2023_tulancingo[['trimestre','segmento']]
df_financiamientos_2023_tulancingo

,trimestre,segmento
0,1,D
1,1,E
2,1,C
3,1,D
4,1,E
5,1,E
6,1,E
7,1,D
8,1,C
9,1,D


In [18]:
# Calcular el total de registros
total_registros = len(df_financiamientos_2023_tulancingo)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(df_financiamientos_2023_tulancingo, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  45
trimestre          1          2          3
segmento                                  
B           2.222222   0.000000   0.000000
C           4.444444   2.222222   6.666667
D          13.333333  15.555556  11.111111
E          20.000000  11.111111  13.333333


In [20]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']
porcentajes = tabla_porcentajes[trimestre]

fig = go.Figure()
trimestres = tabla_porcentajes.columns
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'{trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=porcentajes,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))
fig.update_layout(
    #barmode='stack', 
    #title='Distribución de créditos por segmento (trimestre) 2022',
    yaxis=dict(
        #tickformat='2.2%',
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    legend_title="Trimestre",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_dist_creditos_segmento_%_2023',  carpeta='assets/graficas')
fig.show()
